# Objet : Test pandas

## Objectif

- list vers pandas series

## Résultats
- à préciser

## Usages possibles 
- rempacement _keys et _keys par Series

## Autres points
- à tester

données utilisées : https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/temps-reel/2022/

In [15]:
# derived

# to_json
#keys = idx.derkeys(self.lindex[parent])

def derkeys(self_keys, parent_keys):
        '''return keys derived from parent keys

        *Parameters*

        - **parent** : Field - parent

        *Returns* : list of keys'''
        derkey = [-1] * (max(parent_keys)+1)
        for i in range(len(self_keys)):
            derkey[parent_keys[i]] = self_keys[i]
        if min(derkey) < 0:
            raise FieldError("parent is not a derive Field")
        return derkey

def keysfromderkeys(parentkeys, derkeys):
        '''return keys from parent keys and derkeys

        *Parameters*

        - **parentkeys** : list of keys from parent
        - **derkeys** : list of derived keys

        *Returns* : list of keys'''
        return [derkeys[pkey] for pkey in parentkeys]

parent_keys = [1, 2, 2, 3, 3, 0]
child_keys  = [1, 0, 0, 1, 1, 0]
der_keys = derkeys(child_keys, parent_keys)
print(der_keys)
print(keysfromderkeys(parent_keys, der_keys))

def np_derkeys(self_keys, parent_keys):
        '''return keys derived from parent keys

        *Parameters*

        - **parent** : Field - parent

        *Returns* : list of keys'''
        derkey = np.full([max(parent_keys)+1], -1)
        derkey[parent_keys] = self_keys
        if min(derkey) < 0:
            raise FieldError("parent is not a derive Field")
        return derkey

def np_keysfromderkeys(parentkeys, derkeys):
        '''return keys from parent keys and derkeys

        *Parameters*

        - **parentkeys** : list of keys from parent
        - **derkeys** : list of derived keys

        *Returns* : list of keys'''
        return derkeys[parentkeys]
    
np_der_keys = np_derkeys(child_keys, parent_keys)
print(np_der_keys)
print(np_keysfromderkeys(parent_keys, np_der_keys))

parent_keys = np.array([1, 2, 2, 3, 3, 0])
child_keys  = np.array([1, 0, 0, 1, 1, 0])
np_der_keys = np_derkeys(child_keys, parent_keys)
print(np_der_keys)
print(np_keysfromderkeys(parent_keys, np_der_keys))

[0, 1, 0, 1]
[1, 0, 0, 1, 1, 0]
[0 1 0 1]
[1 0 0 1 1 0]
[0 1 0 1]
[1 0 0 1 1 0]


In [2]:
import numpy as np
from ntv_numpy import Dcomplete, Dfull

data = np.array([10, 20, 30, 30, 40, 30, 50, 50, 30, 10])

values, cat, count = np.unique(data, return_inverse=True, return_counts=True)
print(values, cat, count)
values, idx, cat, count = np.unique(data, return_inverse=True, return_counts=True, return_index=True)
idx_fill = list(count).index(max(count))
#fill_value = values[idx_fill]
sp_index = [row for row, cat in zip(range(len(cat)), cat) if cat != idx_fill] + [idx_fill]
sp_values = list(data[sp_index])
sp_index[-1]= -1
#print(sp_index)
#print(sp_values)
print(Dcomplete(sp_values).to_json())
print(Dfull(sp_values).to_json())

[10 20 30 40 50] [0 1 2 2 3 2 4 4 2 0] [2 1 4 1 2]
[[10, 20, 30, 40, 50], [0, 1, 3, 4, 4, 0, 2]]
[10, 20, 40, 50, 50, 10, 30]


In [ ]:
from ntv_numpy import Dcomplete
cp_values = Dcomplete(sp_values)
print(cp_values.coding)
print(cp_values.data)

In [ ]:
import pandas as pd
from pandas.arrays import SparseArray
arr = SparseArray(data, dtype=pd.SparseDtype("int", fill_value))
print(list(arr.sp_index.indices))
print(list(arr.sp_values))

In [ ]:
from pandas.arrays import SparseArray
arr = SparseArray([0, 0, 10, 20, 0, 0, 10, 30])
# attributs : sp_values, sp_index, fill_values
print(list(arr.sp_index.indices))
print(list(arr.sp_values))
index = list(arr.sp_index.indices)
val = list(arr.sp_values)
values = [0] * 8
for ind, idx in enumerate(arr.sp_index.indices):
    values[idx] = arr.sp_values[ind]
print(values)

In [ ]:
# test pour les tuples
dist = [[len(set(zip(keys[i], keys[j]))) # 9.3
dist = [[len(pd.Series(zip(keys[keys.columns[i]], keys[keys.columns[j]])).astype('category').cat.categories) # 65
dist = [[len(np.unique(np.column_stack((keys[i], keys[j])), axis=0)) # 73.7
dist = [[len(np.unique(np.fromiter(zip(keys[i], keys[j]), dtype='object'))) #146
dist = [[len(keys[[keys.columns[i], keys.columns[j]]].apply(tuple, axis=1).astype('category').cat.categories) # 580

In [ ]:
import json
import shapely
from shapely.geometry import Point, Polygon
import pandas as pd
from json_ntv import ShapelyConnec

data =pd.Series([Point(1,2), Point(1,3), Point(2,4), Polygon([[1.0, 2.0], [1.0, 3.0], [2.0, 4.0]]),
Polygon([[1.0, 2.0], [1.0, 30.0], [30.0, 30.0], [30,2]], [[[5.0, 16.0], [5.0, 27.0], [20.0, 27.0]]])])

coor = data.apply(ShapelyConnec.to_coord)
print(coor)
print(coor.to_list())
data2 = coor.apply(ShapelyConnec.to_geometry)
print(data2)

coor2 = data.apply(shapely.geometry.mapping)
print(coor2)

coor3 = coor2.apply(json.dumps)
print(coor3)

coor4 = coor3.apply(json.loads)
print(coor4)


In [ ]:
coor5 = data.apply(shapely.to_geojson)
coor5

### tests styler

In [ ]:


df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
styler = df.style.highlight_max()

styler

In [ ]:
styler.to_html()

In [ ]:
#styler.to_json() -> ko

### tests numpy

In [ ]:
import numpy as np
from time import time

coef = 40
period = 20
repeat = 50
leng = coef * period * repeat
lseq = coef * period
tn = 0
tp = 0
for i in range(50):
    t0 =time()
    seq = np.stack(tuple([np.arange(period)] * coef), axis=1).reshape(lseq)
    keys1 = list(np.stack(tuple([seq] * repeat)).reshape(leng))
    tn += time()-t0

    t0 =time()
    keys2 = [(i % lseq) // coef for i in range(leng)]
    tp += time()-t0
print(tn/10, tp/10, tp/tn, keys1 == keys2)

------
## Initialisation
- lecture des fichiers de 01/2022 issus de l'api (un fichier par jour)

In [ ]:
from pprint import pprint
from collections import Counter
from time import time
import csv
import os
#os.chdir('C:/Users/a179227/OneDrive - Alliance/perso Wx/ES standard/python ESstandard/ES')
from util import util
from observation import Ilist, Iindex
from copy import copy

chemin='C:/Users/a179227/OneDrive - Alliance/perso Wx/ES standard/Environnemental-Sensing/python/validation/air/data_lcsqa/'

In [ ]:
import pandas as pd
from datetime import date
import json

In [ ]:
sr = pd.Series([date(2021,1,1), date(2022, 1, 2)])
print(sr)
sr2 = sr.astype(str) # conversion date -> str
js = sr2.to_json(orient='records', date_format='iso', default_handler=str)
print(js)

sr3 = pd.read_json(js, typ='series')
sr4 = pd.to_datetime(sr3).dt.date # conversion str -> date
print(sr4.equals(sr))
print(sr4)


In [ ]:
data = []
t0 = time()
annee = 2022
mois = 1
jour = 1
for i in range(4):
    file = chemin + 'FR_E2_' + str(annee) + '-' + format(mois, '02d') +'-' + format(jour+i, '02d') +'.csv'
    data.append(pd.read_csv(file, sep=';'))
data2 = pd.concat(data, ignore_index=True, join='inner').astype('category')
data2.pop('valeur brute')
print('data2', len(data2), list(data2), time()-t0)

In [ ]:
t0=time()
idxs2 = Ilist.obj(data2)
print('idxs (len, lenlidx, sumcodec) : ', len(idxs2), len(idxs2.idxlen), sum(idxs2.idxlen), time()-t0)
#idxs2.delindex('valeur brute')
idxs2.setvar('valeur')

In [ ]:
#calcul de values à partir de keys et codec
t0=time()
for idx in idxs2.lindex:
    val = idx.values
print('list to val old: ', time()-t0)
#génération d'une liste de Series à partir d'une liste de codec
t0=time()
pdcodec=[pd.Series(idx.codec) for idx in idxs2.lindex]
print('list to codec : ', time()-t0)#génération d'une liste de Series à partir d'une liste de keys
#génération d'une liste de Series simple à partir d'une liste de keys
t0=time()
pdkeyss=[pd.Series(idx.keys, dtype='int64') for idx in idxs2.lindex]
print('list to cat simple : ', time()-t0)
#génération d'une liste de Series à partir d'une liste de keys
t0=time()
pdkeys=[pd.Series(idx.keys, dtype='category') for idx in idxs2.lindex]
print('list to cat : ', time()-t0)
#génération d'une liste de Series à partir d'une liste de keys
pdkeys=[]
t0=time()
for idx in idxs2.lindex:
    pdkeys.append(pd.Series(idx.keys, dtype='category'))
print('list to cat : ', time()-t0)#calcul de values à partir de keys et codec avec une Series categorical (default)
t0=time()
for idx in idxs2.lindex:
    val = list(pd.Series(idx.keys, dtype='category').cat.rename_categories(idx.codec))
print('list to val (def): ', time()-t0)
#calcul de values à partir de keys et codec avec une Series map (all)
t0=time()
for idx in idxs2.lindex:
    val = list(pd.Series(idx.keys).map(pd.Series(idx.codec)))
print('list to val (all map): ', time()-t0)
#calcul de values à partir de keys et codec avec une Series (quelconque)
t0=time()
for idx, keys in zip(idxs2.lindex,pdkeys):
    val = list(pd.DataFrame(keys.cat.rename_categories(pd.Series(zip(idx.codec, range(len(idx.codec))))).tolist())[0])
print('list to val (all tuple): ', time()-t0)
#calcul de values à partir d'une Series de keys et codec (default)
t0=time()
for idx, keys in zip(idxs2.lindex,pdkeys):
    val = list(keys.cat.rename_categories(idx.codec))
print('cat to val (def): ', time()-t0)
#calcul de values à partir d'une Series de keys et codec avec une Series (quelconque)
t0=time()
for idx, keys in zip(idxs2.lindex,pdkeys):
    val = list(keys.map(pd.Series(idx.codec)))
print('cat to val (all): ', time()-t0)
#calcul de values à partir d'une Series de keys et une series codec (quelconque)
t0=time()
for cod, keys in zip(pdcodec,pdkeyss):
    val = list(keys.map(cod))
print('simple to val (all series simple): ', time()-t0)
#calcul de values à partir d'une Series de keys et une series codec (quelconque)
t0=time()
for cod, keys in zip(pdcodec,pdkeys):
    val = list(keys.map(cod))
print('cat to val (all series): ', time()-t0)

In [ ]:
#print(val[:100])

In [ ]:
#ESValue
from observation import DatationValue, NamedValue
ser = pd.Series([DatationValue('date'), DatationValue('date2'), DatationValue('date'), NamedValue(3,'test')], dtype='category')
print(ser.cat.categories, '\n', list(ser.cat.codes))
ser2 = pd.Series([0,11,0,22]).astype('category')
serbis = ser2.cat.rename_categories([DatationValue('date'), DatationValue('date2'),  NamedValue(3,'test')])
min(ser.astype('object') == serbis.astype('object'))

In [ ]:
#generation dataframe à partir de Ilist
ilm = Ilist.obj([['plants', ['fruit', 'fruit', 'fruit', 'fruit', 'vegetable', 'vegetable', 'vegetable', 'fruit']],
                  ['quantity', ['kg', '10 kg', 'kg', '10 kg', 'kg', '10 kg', 'kg', '10 kg']],
                  ['product', ['apple', 'apple', 'orange', 'orange', 'peppers', 'peppers', 'banana', 'banana']],
                  ['price', [1, 10, 2, 20, 1.5, 15, 0.5, 5], -1]])
df = pd.concat([idx.to_pandas(index=False, series=False).astype('category') for idx in ilm.lindex], axis=1)
col = pd.MultiIndex.from_arrays([ilm.lname, ilm.lisvar], names=['name', 'var'])
df.columns = col
print(df)

In [ ]:
# fonction append
s2 = pd.Series(['fruit', 'kg', 'banana', 7], index=df.columns)
df = pd.concat([df, s2.to_frame().T], ignore_index=True).astype('category') 
ds = df['price'].squeeze()
ds.cat.categories

In [ ]:
a= pd.Series(['b', 'a', 'a', 'd']).astype('category')
b = pd.Series([1,2,4,3]).astype('category')
coup = pd.Series(list(zip(*(list(a), list(b))))).astype('category')
print(coup)

print(a.cat.codes)
print(coup.cat.codes)
res = coup.cat.codes == a.cat.codes
print(res)

In [ ]:
a = pd.Series([0,1,2,3,2]).astype('category')
codec = ['a', 'b', 'c', 'b']
print(a.cat.codes.map(pd.Series(codec)))

In [ ]:
#filtrage suivant var
df.loc[:,(slice(None),False)]

In [ ]:
full = [[name, list(ds)] for name,ds in df.items()]
print(full, '\n')
default = [[name, list(ds.cat.categories), list(ds.cat.codes)] for name,ds in df.astype('category').items()]
print(default)

In [ ]:
d = {"one": [1.0, 2.0, 3.0, 4.0], "two": [4.0, 3.0, 2.0, 1.0]}
df = pd.DataFrame(d)
df["three"] = True
df["four"] = [True, True, False, False]
df["total"] = True
df['somme'] = 0
f =[1,3]
df.loc[f, "three"] = False
df['total'] = df['total'] & df['three']
df['total'] = df['total'] & df['four']
df['somme'] = df['somme'] - df['three'] +1
df['somme'] = df['somme'] - df['four'] +1
print(df)
print(df.four.sum(), df.total.sum())
maxi = max(df.somme) 
df_max = df[df.somme == maxi]
print(maxi, len(df_max))
print(df_max.loc[:,('one','three', 'four', 'total')])

In [ ]:
print(ilm.to_obj(modecodec='full'), '\n')
print(ilm.to_obj(modecodec='default'))

In [ ]:
#generation de json (full et default)
import json
t0=time()
full = json.dumps([[name, list(ds)] for name,ds in data2.items()])
print('fullsize', len(full), time()-t0)
t0=time()
default = json.dumps([[name, list(ds.cat.categories), list(ds.cat.codes)] for name,ds in data2.astype('category').items()])
print('defaultsize', len(default), time()-t0)


In [ ]:
t0=time()
fullsize = len(idxs2.to_obj(encoded=True, modecodec='full'))
print('fullsize', fullsize, time()-t0)
t0=time()
minsize = len(idxs2.to_obj(encoded=True, modecodec='nokeys'))
print('minsize', minsize, time()-t0)

----
## format non optimisé
- le "taux d'unicité" reste à 12% (pas de modification des index)
- le "taux de codage" est de 30% (remplacement des données dupliquées par un entier)
- le gain de taille de fichier par rapport à un fichier "quoté" est de 61%
- l'analyse de la structure montre que les données sont principalement du type "linked" (non ou peu structuré)
- quelques colonnes sont de type "derived". Par exemple les index longitude(43) et latitude(44) sont bien dérivés de l'index coordonneesXY(13)
- le taux de couplage ("linkrate") pour chacun des index est très proche de 0, ce qui signifie que les données devraient être de type "derived" (lien de dépendance par exemple comme entre les trimestres et les mois) ou "coupled" (lien biunivoque).

In [ ]:
t0=time()
defaultsize = len(idxs2.to_obj(encoded=True, modecodec='default'))
print('defaultsize', defaultsize, time()-t0)
print('indicator default : ', idxs2.indicator(fullsize, defaultsize))
pprint(idxs2.indexinfos(keys=['num', 'name', 'lencodec', 'parent', 'typecoupl']), width=120)
pprint(idxs2.indexinfos(keys=['linkrate']))

----
## Format optimisé
- le "taux d'unicité" se dégrade légèrement (passage de 11,6% à 12,1%) par l'ajout d'index supplémentaires
- le "taux de codage" par contre passe de 30% à 16% de par l'optimisation 
- le gain de taille de fichier par rapport à un fichier "quoté" est maintenant de 74%
- l'utilisation d'un format binaire (codage CBOR pour Concise Binary Object Representation RFC 8949) permet d'améliorer encore le gain de taille de fichier (82%)

In [ ]:
#idxs.setcanonorder().sort()
t0=time()
optimizesize = len(idxs2.to_obj(modecodec='optimize', encoded=True))
print('optimizesize ', optimizesize, time()-t0)
print('indicator optimize : ', idxs2.indicator(fullsize, optimizesize))
t0=time()
js = idxs2.to_obj(encoded=True, modecodec='optimize', format='cbor')
cborsize = len(js)
print('cborsize', cborsize, time()-t0)
print('indicator cbor : ', idxs2.indicator(fullsize, cborsize))

----
## Intégrité
- la transformation inverse des données binaires permet de vérifier qu'on retombe bien sur les mêmes données (pas de dégradation)

In [ ]:
t0=time()
idxs3 = Ilist.from_obj(js)
print('fromcbor', len(idxs3), time()-t0)
t0=time()
verif = idxs2 == idxs3
print('controle égalité :', verif, time()-t0)
